<a href="https://colab.research.google.com/github/deanzedd/Machine-Learning/blob/main/KNN_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# khoảng cách điểm dữ liệu

code mẫu phần tính khoảng cách các điểm dữ liệu

In [ ]:
from __future__ import print_function
import numpy as np
from time import time #for comparing running time
d, N=1000, 10000 #dimension and number of training points
X=np.random.randn(N,d) #N d-dimensional points
z=np.random.randn(d) #d-dimensional vector


In [ ]:
#naively compute square distance between two vector(point to point)
def dist_pp(z,x) :
  d = z - x.reshape(z.shape) # force x and z to have the same dims
  return np.sum(d*d)

## ps = point to set
#from one point to each point in a set, naive
def dist_ps_naive(z,X) :
  N = X.shape[0] #N = N(N rows in X)
  res = np.zeros((1,N))
  for i in range(N) :
    res[0][i] = dist_pp(z,X[i])
  return res

# from one point to each point in a set, fast
def dist_ps_fast(z,X) :
  X2 = np.sum(X*X,1) # square of l2 norm of each X[i], can be precomputed
  z2 = np.sum(z*z) # square of l2 norm of z
  return X2 + z2 - 2*X.dot(z) # z2 can be ignored

t1 = time()
D1 = dist_ps_naive(z,X)
print('naive point2set, running time:', time() - t1, 's')

t1 = time()
D2 = dist_ps_fast(z,X)
print('fast point2set , running time:', time() - t1, 's')
print('Result difference:', np.linalg.norm(D1 - D2))



naive point2set, running time: 0.16475605964660645 s
fast point2set , running time: 0.06168937683105469 s
Result difference: 2.3863406147372906e-11


In [ ]:
Z=np.random.randn(100,d) #100 d-dimesional points
#ss= set to set
#from each point in one set to each point in another set, half fast
def dist_ss_0(Z,X) :
  M,N = Z.shape[0],X.shape[0]
  res = np.zeros((M,N))
  for i in range(M) :
    res[i] = dist_ps_fast(Z[i],X)
  return res
print(Z.shape) #(100,1000=d)
print(X.shape[0]) #10000=N

#from each point in one set to each point in another set, fast
def dist_ss_fast(Z,X) :
  X2 = np.sum(X*X,1) # square of l2 norm of each row of X
  Z2 = np.sum(Z*Z,1) # square of l
  return Z2.reshape(-1,1) + X2.reshape(1,-1) - 2*Z.dot(X.T)

t1 = time()
D3 = dist_ss_0(Z,X)
print('half fast set2set running time:', time() - t1, 's')
t1 = time()
D4 = dist_ss_fast(Z,X)
print('fast set2set running time:', time() - t1, 's')
print('Result difference:', np.linalg.norm(D3 - D4))

(100, 1000)
10000
half fast set2set running time: 6.827006101608276 s
fast set2set running time: 0.12366271018981934 s
Result difference: 1.008814233156002e-10


# IRis flower data set

In [ ]:
from __future__ import print_function
import numpy as np
from sklearn import neighbors, datasets
from sklearn.model_selection import train_test_split #for splitting data
from sklearn.metrics import accuracy_score #for evaluating results

iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
print('Labels:', np.unique(iris_y))

# split train and test
np.random.seed(7) #7 thêm vào để đảm bảo kết quả chạy(7 có thể thay bằng số bất kì 32 bit)
X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size=130)

print('Train size:', X_train.shape[0], ', test size:', X_test.shape[0])


Labels: [0 1 2]
Train size: 20 , test size: 130


*result of 1NN*

In [ ]:
#result of 1NN
model = neighbors.KNeighborsClassifier(n_neighbors = 1, p = 2) #K=1,p=2 chính là l2 norm(p=1 là l1 norm)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy of 1NN: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Accuracy of 1NN: 92.31 %


*result of 7NN*

In [ ]:
model = neighbors.KNeighborsClassifier(n_neighbors= 7, p = 2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy of 7NN with major voting: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Accuracy of 7NN with major voting: 93.85 %


*đánh trọng số điểm lân cận*

điểm càng gần thì càng tốt-> trọng số cao
weights = ’distance’

weights là ’uniform’ (default)


In [ ]:
model = neighbors.KNeighborsClassifier(n_neighbors= 7, p=2, weights='distance')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy of 7NN (1/distance weights): %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Accuracy of 7NN (1/distance weights): 94.62 %


*KNN tự định nghĩa*

In [ ]:
def myweight(distances) :
  sigma2 = .4 # we can change this number
  return np.exp(-distances**2/sigma2)

model = neighbors.KNeighborsClassifier(n_neighbors= 7, p=2, weights=myweight)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy of 7NN (customized weights): %.2f %%" %(100*accuracy_score(y_test, y_pred)))
#

Accuracy of 7NN (customized weights): 95.38 %


#KNN from scratch

với tập dữu liệu iris flower

In [92]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from scipy import stats
import matplotlib.pyplot as plt
from time import time # tính toán thời gian thực hiện

In [93]:
data = load_iris()
df = pd.DataFrame(data = data.data, columns= data.feature_names )
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [94]:
X,y = data.data, data.target
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=130,random_state=42)
print(X_train.shape[0])


20


In [95]:
class KNN1():
  def __init__(self,k=2, metric = 'euclidean', p=None):
    self.k =k
    self.metric = metric
    self.p = p

  def fit(self, X, y):
    self.X_train = X
    self.y_train = y

  def euclidean(self, v1, v2):
    return np.sqrt(np.sum((v1-v2)**2))

  def manhattan(self, v1, v2):
    return np.sum(np.abs(v1-v2))

  def predict(self, X_test):
    preds = []
    for test_row in X_test:
      nearest_neighbours = self.get_neighbours(test_row)
      majority = stats.mode(nearest_neighbours).mode #tìm 1 giá trị xuất hiện nhiều nhất với đầu vào nearest_neighbours
      preds.append(majority)
    return preds

  def get_neighbours(self, test_row):
    dis = list()

    for(train_row, train_class) in zip(self.X_train, self.y_train): #zip(self.X_train, self.y_train) ghép 2 thành phần của X_train và y_train thành 1 cặp
      if self.metric == 'euclidean':
        dist = self.euclidean(train_row, test_row)

      elif self.metric == 'manhattan':
        dist = self.manhattan(train_row, test_row)

      else:
        raise NameError('Supported metrics are euclidean, manhattan')

      dis.append((dist, train_class))

    dis.sort(key=lambda x: x[0]) # sắp xếp lại dis dựa trên thành phần đầu tiên của mỗi phần tử theo thứ tự tăng dần(ở ví dụ này nó là khoảng cách của test_row -> từng điểm trong tập train)
    neighbours = list() # neighbours là 1 list k phần tử chứa các loại train_class
    for i in range(self.k):
      neighbours.append(dis[i][1]) #
    return neighbours


In [96]:
for metric in ['euclidean', 'manhattan']:
  clf = KNN1(k=7, metric=metric)
  clf.fit(X_train, y_train)
  t1 = time()
  preds = clf.predict(X_test)
  accuracy = accuracy_score(y_test, preds)
  print(f'Metric: {metric}, accuracy: {accuracy}, time: {time() -t1}')


Metric: euclidean, accuracy: 0.9615384615384616, time: 0.21751904487609863
Metric: manhattan, accuracy: 0.9538461538461539, time: 0.16148710250854492


thử trên tập MNIST

In [99]:
class KNN2():
  def __init__(self,k=2, metric = 'euclidean', p=None):
    self.k =k
    self.metric = metric
    self.p = p

  def fit(self, X, y):
    self.X_train = X
    self.y_train = y

  def euclidean(self, v1, v2):
    return np.sqrt(np.sum((v1-v2)**2))

  def manhattan(self, v1, v2):
    return np.sum(np.abs(v1-v2))

  def predict(self, X_test):
    preds = []
    for test_row in X_test:
      nearest_neighbours = self.get_neighbours(test_row)
      majority = stats.mode(nearest_neighbours).mode #tìm 1 giá trị xuất hiện nhiều nhất với đầu vào nearest_neighbours
      preds.append(majority)
    return preds

  def get_neighbours(self, test_row):
    dis = list()

    for(train_row, train_class) in zip(self.X_train, self.y_train): #zip(self.X_train, self.y_train) ghép 2 thành phần của X_train và y_train thành 1 cặp
      if self.metric == 'euclidean':
        dist = self.euclidean(train_row, test_row)

      elif self.metric == 'manhattan':
        dist = self.manhattan(train_row, test_row)

      else:
        raise NameError('Supported metrics are euclidean, manhattan')

      dis.append((dist, train_class))

    dis.sort(key=lambda x: x[0]) # sắp xếp lại dis dựa trên thành phần đầu tiên của mỗi phần tử theo thứ tự tăng dần(ở ví dụ này nó là khoảng cách của test_row -> từng điểm trong tập train)
    neighbours = list() # neighbours là 1 list k phần tử chứa các loại train_class
    for i in range(self.k):
      neighbours.append(dis[i][1]) #
    return neighbours

In [101]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version =1, as_frame= False)
X,y = mnist.data, mnist.target
y = y.astype(int)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_test = X[:8000], X[8000:10000]
y_train, y_test = y[:8000], y[8000:10000]
clf = KNN2(k=7, metric= 'euclidean')
clf.fit(X_train, y_train)
t1 = time()
preds = clf.predict(X_test)
accuracy = accuracy_score(y_test, preds)
print(f'accuracy: {accuracy}, time: {time() -t1}')



accuracy: 0.925, time: 261.2455406188965


# TỔNG KẾT

ƯU ĐIỂM
- độ phức tạp thấp(do tính toán nhiều ở pha huấn luyện)
- việc dự đoán kết quả đơn giản khi xác định được các lân cận
- KNN ko cần giả sử phân phối của từng nhãn

NHƯỢC ĐIỂM
- KNN nhạy cảm khi K nhỏ
- KNN phần lớn tính toán ở pha kiểm tra.
trong đó phần lớn tính toán ở các điểm dữ liệu huấn luyện nên sẽ tốn nhiều thời gian